# 装饰器
## 理解装饰器

理解装饰器，我们必须要理解当我们给一个函数加上装饰器的时候，真正发生了什么，比如下面的代码。

```python
@my_decorator
def my_func(*args, **kwargs):
    pass
```

实际上这一段代码在编译的时候，编译器真正执行的是
```python
def my_func(*args, **kwargs):
    pass
my_func = my_decorator(my_func)
```
简单来说，`my_func`函数先被定义，然后又被`my_decorator(my_func)`的返回值重新定义了。现在来看`my_decotator`:

- 首先它必须是一个可调用的对象，最常见的，必须是一个函数。它也可以是一个类，此为后话。
- 其次它必须且只能接受一个参数`my_fun`。
- 它必须有一个返回值，这个返回值用来覆盖原来的`my_fun`。按常理，返回值应该也是一个函数，只有这样，`my_fun`才能继续被当成函数调用。其实也不尽然，此为另一后话。

现在我们尝试自己写一个装饰器。


In [ ]:
from __future__ import print_function
from __future__ import unicode_literals


def my_decorator(func): #接受一个函数参数
    def wrapper(*args, **kwargs):
        print("before {} ...".format(func.__name__))
        return_var = func(*args, **kwargs)
        print("After {} ...".format(func.__name__))
        return return_var
    return wrapper
@my_decorator
def my_func():
    """print myself"""
    print("In my_func")

my_func()      

可以看到，在上述代码，我们在`my_decorator`里面定义了一个函数`wrapper`，`wrapper`接受任何参数，并将其传递给func并获得func的返回值。`my_decorator`的返回值为`wrapper`函数，用来覆盖`my_func`。此时打印`my_func`的名字，获得的是`wrapper`。

In [ ]:
print(my_func.__name__) #wrapper
print(my_func.__doc__) #None

正常情况下我们希望这些函数的性质，比如函数名，docstring等能够保留下来，此时需要`functools.wraps`：

In [ ]:
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("before {} ...".format(func.__name__))
        return_var = func(*args, **kwargs)
        print("After {} ...".format(func.__name__))
        return return_var
    return wrapper
@my_decorator
def my_func():
    """print myself"""
    print("In my_func")

my_func()
print("function name is:", my_func.__name__)
print("function docstring is:", my_func.__doc__)